# Geochronology Calculations

In [1]:
import matplotlib.pyplot as plt
from bokeh.plotting import figure, output_notebook, show
from bokeh.layouts import column
from bokeh.models import Range1d, LinearAxis, ColumnDataSource, LabelSet, Span, Slope, Label, Legend

from scipy.interpolate import CubicSpline
import pandas as pd
import numpy as np
from IPython.core.display import display, HTML
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
output_notebook()

import geochron_apps as gc

Loading BokehJS ...

<center><img src="images/geochronology.png" align="center">
https://www.explainxkcd.com/wiki/index.php/1829:_Geochronology
</center>

The following presentation shows some of the geochronology calculations learned in the Advanced Geochronology class at University of Saskatchewan taught by Bruce Eglington and Camille Partin, 2021. Some of the images in this presentation are taken from lectures given by the instructor.

This notebook contains sample calculations typically used in geochronology. It can be obtained at https://git.cs.usask.ca/msv275/advanced-geochronology.  
It can be cloned through the git command:  
* git clone https://git.cs.usask.ca/msv275/advanced-geochronology.git


## U-Pb Calculations

* Calculate and graph a Wetherill Concordia diagram assuming the 1975 IUGS constants
* Calculate and graph a Tera-Wasserburg Concordia diagram assuming the 1975 IUGS constants
* Calculate the 206Pb/238U, 207Pb/235U and 207Pb/206Pb ratios for a zircon formed at 3000 Ma, 2000 Ma, 1000 Ma, 500 Ma, 200 Ma and 100 Ma and plot these on both of the above graphs
* Graph Concordia curves on both Wetherill and Tera-Wasserburg plots using the most recently published decay constants and ratios for 238U and 235U decay. 
* Compare the old and new decay constant and ratio results

First we need to calculate the ratios needed for each plot. The latest decay rate I could get were derived from the half life of 235U = 703.05(+-0.58) Ma from Mattinson(2010). For 238U = 4.4683(+-0.0096) Ga from Villa et al. (2016).  
The equation used to calculate this decay constant is: t<sub>1/2</sub> = 0.693/*k*

In [2]:
lambda238=1.55125*10**-10,
lambda235=9.8485*10**-10
lambda238_2016=gc.convert_halflife(4469.30)
lambda235_2010=gc.convert_halflife(703.05)
print("The decay rate for U238 in 2016 was measured at {}.".format(lambda238_2016))
print("The decay rate for U235 in 2010 was measured at {}.".format(lambda235_2010))

The decay rate for U238 in 2016 was measured at 1.5505783903519566e-10.
The decay rate for U235 in 2010 was measured at 9.857051418818007e-10.


In order to plot both the Wetherill and Tera-Wasserburg plots we need to calculate some U and Pb isotope ratios.  
* For the Wetherill Concordia we need 206Pb/235U and 207Pb/238U.
* For the Tera-Wasserburg we need 207Pb/206Pb and 238U/206Pb  
The equation used to calculate ratio of daughter to parent is *e*<sup>λ x t</sup> - 1.  This is used for the 207Pb/235U and 206Pb/238U ratios.  
The inverse of this equation is used to calculate 238U/206Pb.
Finally the 207Pb/206Pb is already known from the previous calculations, however it must be multiplied by 1/137.88 to account for the assumed 238U to 235 ratio in all rocks.  
This will be done using the 1975 and the most recent data seperately.

In [3]:
df_1975 = gc.get_U_ratio_data()
df_2020 = gc.get_U_ratio_data(lambda238_2016,lambda235_2010)

As a sanity check, we will print off the first 5 calculations from each set.

In [4]:
df_1975.head()

,age,ratio_238U_206Pb,ratio_207Pb_235U,ratio_206Pb_238U,ratio_207Pb_206Pb
0,0.001,6.446414e+06,9.848505e-07,1.551250e-07,0.046046
1,1.000,6.445914e+03,9.853351e-04,1.551370e-04,0.046065
2,2.000,3.222707e+03,1.971641e-03,3.102981e-04,0.046084
3,3.000,2.148305e+03,2.958919e-03,4.654833e-04,0.046103
4,4.000,1.611104e+03,3.947170e-03,6.206925e-04,0.046122


In [5]:
df_2020.head()

,age,ratio_238U_206Pb,ratio_207Pb_235U,ratio_206Pb_238U,ratio_207Pb_206Pb
0,0.001,6.449206e+06,9.857056e-07,1.550579e-07,0.046105
1,1.000,6.448706e+03,9.861911e-04,1.550699e-04,0.046125
2,2.000,3.224103e+03,1.973355e-03,3.101638e-04,0.046144
3,3.000,2.149235e+03,2.961492e-03,4.652817e-04,0.046163
4,4.000,1.611802e+03,3.950604e-03,6.204237e-04,0.046182


In [6]:
age_list = [100,200,500,1000,2000,3000,4000,4500]

ages = gc.get_age_df(df_2020,age_list)
ages

,age,ratio_238U_206Pb,ratio_207Pb_235U,ratio_206Pb_238U,ratio_207Pb_206Pb
100,100.0,63.993356,0.103592,0.015627,0.048080
200,200.0,31.748616,0.217916,0.031497,0.050178
500,500.0,12.404873,0.636979,0.080613,0.057308
1000,1000.0,5.962123,1.679701,0.167725,0.072633
2000,2000.0,2.750405,6.180796,0.363583,0.123293
3000,3000.0,1.688361,18.242386,0.592290,0.223381
4000,4000.0,1.163659,50.563836,0.859358,0.426741
4500,4500.0,0.990837,83.408925,1.009248,0.599396


The next plot uses the bokeh library and is interactive. It will allow us to zoom and pan and look at the lines in more detail.

In [7]:
figure1 = gc.get_figure("Wetherill Concordia Diagram", "207Pb/235U", "206Pb/238U", [0,100], [0,1.2])
figure1 = gc.plot_wetherill(figure1, df_1975, "green", "U Decay Rate 1975", 6)
figure1 = gc.plot_wetherill(figure1, df_2020, "red", "U Decay Rate Recent", 2)
figure1 = gc.plot_ages_wetherill(figure1, ages)
figure1.legend.location = "bottom_right"

In [8]:
show(figure1)

This plot shows that there is very little differentiation, at a large scale. If we zoom in, we can see some seperation. Now we will do the same with the Tera-Wasserburg plot.

In [9]:
figure2 = gc.get_figure("Tera-Wasserburg Concordia Diagram", "238U/206Pb", "207Pb/206Pb", [0,75], [0,0.62])
figure2 = gc.plot_tera_wasserburg(figure2, df_1975, "green", "U Decay Rate 1975", 6)
figure2 = gc.plot_tera_wasserburg(figure2, df_2020, "red", "U Decay Rate Recent", 2)
figure2 = gc.plot_ages_tera_wasserburg(figure2, ages)
figure2.legend.location = "top_right"

In [10]:
show(figure2)